# Refactored notebook for modelling

## imports

In [ ]:
import sys

sys.path.append("..")

import numpy as np
import pandas as pd
import warnings
import re

from NHS_PROMs.load_data import load_proms, structure_name
from NHS_PROMs.preprocess import filter_in_range, filter_in_labels, method_delta
from NHS_PROMs.utils import downcast, map_labels, fillna_categories, pd_fit_resample
from NHS_PROMs.data_dictionary import meta_dict

# use adjusted fillna which can cope with non-existing categories
pd.core.frame.DataFrame.fillna = fillna_categories
pd.core.frame.DataFrame.fillna = fillna_categories

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_selector
# from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.inspection import permutation_importance
from sklearn import set_config
set_config(display='diagram')

from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline, make_pipeline

# enable autodetect by using "infer" + the use of column names
SMOTENC.fit_resample = pd_fit_resample(SMOTENC.fit_resample)

## load data
General approach is not DRY for the sake of availability of having knee and hip df's always at hand, but also keep it readable (script-wise).

In [ ]:
# load data + rename columns with structired name
# df_knee_raw = load_proms(part="knee").apply(downcast).rename(structure_name, axis=1)
df_hip_raw = load_proms(part="hip").apply(downcast).rename(structure_name, axis=1)

# get meta data for each
full_meta = {t + k: v for k, v in meta_dict.items() for t in ["t0_", "t1_"]}
hip_meta = {k: v for k, v in full_meta.items() if k in df_hip_raw.columns}

df_hip_raw.sample(3)

## basic cleaning

In [ ]:
endings = (
    "code",
    "procedure",
    "revision_flag",
    "assisted_by",
    "profile",
    "predicted",
)
cols2drop = [c for c in df_hip_raw.columns if c.endswith(endings)]

In [ ]:
%%time
df_hip_clean = (
    df_hip_raw.apply(lambda s: filter_in_range(s, **hip_meta[s.name]))
    .apply(lambda s: filter_in_labels(s, **hip_meta[s.name]))
    .apply(lambda s: map_labels(s, **hip_meta[s.name]))
    .query("t0_revision_flag == 'no revision'")
    .drop(columns=cols2drop)
    #     .replace("missing", np.nan)
)

df_hip_clean.sample(3)

## split data

In [ ]:
# split train + test set
# df_knee_seen = df_knee_clean.query("t0_year != '2019/20'")
# df_knee_unseen = df_knee_clean.query("t0_year == '2019/20'")

df_hip = df_hip_clean.query("t0_year != '2019/20'")
df_hip_unseen = df_hip_clean.query("t0_year == '2019/20'")

df_hip.sample(3)

## create delta dataframes

In [ ]:
# df_org = df_hip_seen.apply(
#     lambda s: map_labels(s, backwards=True, **hip_meta[s.name])
# ).apply(np.asarray)

# # df_knee_delta = method_delta(df_knee_train)
# df_hip_delta = method_delta(df_org)

# # now you could join them again with the original df ...
# # eg: df_hip_train.join(df_hip_delta)
# df_hip_delta.sample(5)

## Make feature set

In [ ]:
# asses quickly missing
print(len(df_hip), "original")
print(len(df_hip.dropna()), "after possible total dropna")
(df_hip.isna().sum() / len(df_hip)).sort_values(ascending=False).head(10)

In [ ]:
# remove NaNs from non categorical/ordinal columns (numerical)
print(len(df_hip), "original")
num_cols = df_hip.select_dtypes(exclude="category").columns
df_hip = df_hip.dropna(subset=num_cols).fillna(value="missing")

print(len(df_hip), "after dropna on numerical + fillna on categories")

In [ ]:
# create x, y
X = df_hip.filter(regex="t0")
y = (df_hip["t1_ohs_score"] - df_hip["t0_ohs_score"] <= 20).astype(int) # knee <= 7

# make a smaller selection of our training data to play with
X = X.iloc[:1000, -5:]
y = y.iloc[:1000]


# create train, test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42, stratify=y
)

## make balanced

In [ ]:
# print("before:")
# display(y_train.value_counts())

# # cat_cols = X_train.dtypes == "category"
# # cat_cols = X_train.columns[cat_cols] TO DO: Fix with eg column selector

# resampler = SMOTENC(categorical_features="infer")
# X_train_balanced, y_train_balanced = resampler.fit_resample(X_train, y_train)

# print("after:")
# display(y_train_balanced.value_counts())

## make + train a simple pipeline

In [ ]:
# make the pipeline
# ct = make_column_transformer(
#     (OneHotEncoder(), make_column_selector(dtype_include="category")),
#     (StandardScaler(), make_column_selector(dtype_include="number")),
# )
#
# pl = make_pipeline(
# #     SMOTENC(categorical_features="infer"), # disabled just for the sake of speed
#     ct,
#     KNeighborsClassifier(),
# )


ct = ColumnTransformer(
    (
        ("categorical", OneHotEncoder(), make_column_selector(dtype_include="category")),
        ("numerical", StandardScaler(), make_column_selector(dtype_include="number")),
    ),
    remainder="drop",
)

pl = Pipeline(
    (
        ("balancer", SMOTENC(categorical_features="infer")),
        ("by_column_types", ct),
        ("model", KNeighborsClassifier()),
    )
)

# train the pipeline/model
pl.fit(X_train, y_train)

## predict + evaluate

In [ ]:
# make prediction
y_hat = pl.predict(X_test.head(500))

# evaluate
print(classification_report(y_test.head(500), y_hat))

## Gridsearch

In [ ]:
# create parameter grid to search on 
# standard (same as pipeline)
param_grid = dict()

# construct gridsearch
GS = GridSearchCV(pl, param_grid=param_grid) ## add scoring

# train gridsearch
GS.fit(X_train, y_train)

# show results
pd.DataFrame(GS.cv_results_)\
    .filter(regex=r"^(?!.*(split|time)).*$")\
    .set_index("rank_test_score").sort_index()

In [ ]:
# create parameter grid to search on 

# standard same as pipeline
param_grid = dict()

# # two models with default parameters
# param_grid = {"model": [KNeighborsClassifier(), DecisionTreeClassifier()]}

# # tuning hyper parameters
# param_grid = {
#     "model": [RandomForestClassifier(), AdaBoostClassifier()],
#     "model__n_estimators": [25, 50, 100],
# }

# # tuning different hyper parameters on different models
# param_grid = [
#     {
#         "model": [RandomForestClassifier()],
#         "model__n_estimators": [25, 50, 100],
#     },
#     {
#         "model": [KNeighborsClassifier()],
#         "model__n_neighbors": [2, 5, 10],
#     },
# ]


# construct gridsearch

# standard
GS = GridSearchCV(pl, param_grid=param_grid)

# # add scoring 
# GS = GridSearchCV(pl, param_grid=param_grid, scoring=["f1"])

# # multiple
# GS = GridSearchCV(pl, param_grid=param_grid, scoring=["balanced_accuracy", "f1"], refit=False)


# train gridsearch
GS.fit(X_train, y_train)

# show results
pd.DataFrame(GS.cv_results_)\
    .filter(regex=r"^(?!.*(split|time)).*$")
#     .set_index("rank_test_score").sort_index()

## regression

In [ ]:
# create x, y
X = df_hip.filter(regex="t0")
y = df_hip["t1_ohs_score"] - df_hip["t0_ohs_score"]


# create train, test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

# make a smaller selection of our training data to play with
X_train = X_train.iloc[:1000, -5:]
y_train = y_train.iloc[:1000]

In [ ]:
# make parameter grid
param_grid = {
    "balancer": ["passthrough"],
    "model": [KNeighborsRegressor()],
}

GS = GridSearchCV(pl, param_grid=param_grid)
# train gridsearch
GS.fit(X_train, y_train)

# show results
pd.DataFrame(GS.cv_results_)\
    .filter(regex=r"^(?!.*(split|time)).*$")
#     .set_index("rank_test_score").sort_index()

## extract feature names pl

In [ ]:
# get the feature names from pipeline
def get_feature_names(sklobj, feature_names=[]):

    if isinstance(sklobj, Pipeline):
        for name, step in sklobj.steps:
            get_feature_names(step, feature_names)
    elif isinstance(sklobj, ColumnTransformer):
        for name, transformer, columns in sklobj.transformers_:
            feature_names += get_feature_names(transformer, columns)
    elif isinstance(sklobj, OneHotEncoder):
        feature_names = sklobj.get_feature_names(feature_names).tolist()
    elif isinstance(sklobj, str):
        if sklobj == "passthrough":
            pass
        elif sklobj == "drop":
            feature_names = []
            
    return feature_names

In [ ]:
get_feature_names(pl)

In [ ]:
# # this is slow ...
# r = permutation_importance(pl, X_train.head(1_000), y_train.head(1_000), n_repeats=2, random_state=0)

# feature_names = get_feature_names(pl)

# for i in r.importances_mean.argsort()[::-1]:
#     if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
#         print(f"{feature_names[i]:<8}"
#         f"{r.importances_mean[i]:.3f}"
#         f" +/- {r.importances_std[i]:.3f}")

## a more advanced pipeline

In [ ]:
# TO DO ...